In [ ]:
import sys
sys.path.append('/home/silja/fatepredict/fatepredict_experiments')

In [5]:
import zarr
import numpy as np
import networkx as nx
from skimage.measure import regionprops
from segmentation import segment_stats
from linajea_cost_test import get_merge_graph_from_array
from funlib.math import decode64

In [15]:
z = zarr.open('anno_alice_T2030_tiny.zarr','r')
gt_image = z['gt_trackimage'][:]
raw = z['Raw'][:]

In [16]:
# annation label is 10000, 10001, 10004, 10005, 10007, 10010, 10013, 10016, 10017
gt_mask = gt_image >=10000
gt_image = gt_mask * gt_image

In [14]:
# get the id list of one time frame
id, positions, _ = segment_stats(gt_image[0],0)

array([    0, 10000, 10001, 10004, 10005, 10007, 10010, 10013, 10016,
       10017], dtype=uint64)

In [ ]:
t = 0
merge_tree = z['Merge_tree/Merge/'+str(t)]
scores = z['Merge_tree/Scoring/'+str(t)]
merge_tree = get_merge_graph_from_array(merge_tree,scores)

# count the number of how many annotation id in merge tree
def count_anno_id(id,merge_tree):
    '''
    merge_tree: nx.Digrph merge_tree create by waterz 
    id: ndarray (n,) 1d array store all ids in GT image
    correct_id_num: int count the number of how many annotation node(id) in merge tree
    '''
    mt_id = list(merge_tree.nodes)
    correct_id_num = len(set(id)&set(mt_id))
    return correct_id_num 

correct_id_num = count_anno_id(id,merge_tree)


In [ ]:
import napari
viewer = napari.Viewer()
viewer.add_image(raw[3])
viewer.add_labels(gt_image)

In [ ]:
# read .gexf file which created by run_candidate_graph.ipynb
graph = nx.read_gexf('text.gexf')

In [ ]:
def get_select_graph(graph):
    select_graph = nx.DiGraph()
    for u,v,data in graph.edges(data=True):
        if data['selected']:
            select_graph.add_edge(v, u, source = v, target = u)
            select_graph.add_node(u, **graph.nodes[u])
            select_graph.add_node(v, **graph.nodes[v])
    return select_graph

In [ ]:
select_graph = get_select_graph(graph)

In [ ]:
def pick_select_edge(graph):
    edges = []
    selected_edges = []
    for u,v,data in graph.edges(data=True):
        edges.append((u,v))
        if data['selected']:
            selected_edges.append((u,v))
    return edges, selected_edges

edges, selected_edges = pick_select_edge(graph)

In [ ]:
def selected_nodes_t(graph,t):
    nodes_t = []
    for node in graph.nodes:
        cor = decode64(int(node),dims=5,bits=[9,12,12,12,19])
        if cor[0] == t:
            nodes_t.append(node)
    return nodes_t

In [ ]:
track_data = []
# creat a list for tracking node
track_id_node = []
track_id = 0
for node in select_graph.nodes:
    cor = decode64(int(node),dims=5,bits=[9,12,12,12,19])
    track_data.append([track_id,cor[0],cor[1],cor[2],cor[3]])
    track_id_node.append(node)
    track_id += 1

In [ ]:
from candidate_graph import iterate_tree
def add_select_nodes_mask(z):
    time = 3
    cell = np.zeros(fragments.shape,dtype=int)
    for t in range(time):
        ids = z['Fragment_stats/id/'+str(t)]
        merge_tree = z['Merge_tree/Merge/'+str(t)]
        scores = z['Merge_tree/Scoring/'+str(t)]
        merge_tree = get_merge_graph_from_array(merge_tree,scores)
        for node in select_graph.nodes:
            if merge_tree.has_node(int(node)):
                # find the merge path
                sub_a = iterate_tree(merge_tree,int(node))
                # creata new cell mask for node
                mask = np.zeros(fragments[t].shape)
                #find id
                for a in sub_a:
                    if a in list(ids):
                        # a is the leave node id
                        # get the label of this leave node
                        label_a = list(ids).index(a)+1
                        mask[fragments[t] == label_a] = 1
                cell[t][mask==1] = track_id_node.index(node)
        return cell

In [ ]:
cell = add_select_nodes_mask(z)